In [3]:
# --- WORK-browser för Jupyter (ingen databas krävs) ---
import pandas as pd
import numpy as np
from IPython.display import display, clear_output
import ipywidgets as w

# 1) En liten WORK-klass (vår "SAS WORK")
class Work:
    def __init__(self):
        self._tables = {}
    def add(self, name, df):
        if not isinstance(df, pd.DataFrame):
            raise TypeError("WORK tar bara pandas.DataFrame")
        self._tables[name] = df
    def get(self, name):
        return self._tables[name]
    def names(self):
        return list(self._tables.keys())
    def overview(self):
        return pd.DataFrame({
            "Name": list(self._tables.keys()),
            "Rows": [len(df) for df in self._tables.values()],
            "Cols": [df.shape[1] for df in self._tables.values()],
        })
    def remove(self, name):
        self._tables.pop(name, None)

WORK = Work()

# 2) Demo: skapa ~10 tabeller + någon join (så du ser hur det funkar)
np.random.seed(0)

WORK.add("customers", pd.DataFrame({
    "cust_id": range(1, 11),
    "name": [f"Customer_{i}" for i in range(1, 11)],
    "city": np.random.choice(["Stockholm", "Göteborg", "Malmö"], size=10)
}))

WORK.add("products", pd.DataFrame({
    "prod_id": range(101, 106),
    "product": ["Laptop", "Phone", "Tablet", "Headset", "Monitor"],
    "price": [12000, 8000, 5000, 1200, 3000]
}))

WORK.add("orders", pd.DataFrame({
    "order_id": range(1001, 1011),
    "cust_id": np.random.choice(WORK.get("customers")["cust_id"], size=10),
    "prod_id": np.random.choice(WORK.get("products")["prod_id"], size=10),
    "quantity": np.random.randint(1, 5, size=10)
}))

WORK.add("payments", pd.DataFrame({
    "payment_id": range(2001, 2011),
    "order_id": np.random.choice(WORK.get("orders")["order_id"], size=10),
    "amount": np.random.randint(1000, 20000, size=10),
    "method": np.random.choice(["Card", "Swish", "Invoice"], size=10)
}))

WORK.add("suppliers", pd.DataFrame({
    "supp_id": range(301, 306),
    "supplier": [f"Supplier_{i}" for i in range(1, 6)],
    "city": np.random.choice(["Stockholm", "Göteborg", "Malmö"], size=5)
}))

WORK.add("inventory", pd.DataFrame({
    "prod_id": WORK.get("products")["prod_id"],
    "stock": np.random.randint(5, 50, size=5),
    "supp_id": np.random.choice(WORK.get("suppliers")["supp_id"], size=5)
}))

WORK.add("employees", pd.DataFrame({
    "emp_id": range(401, 406),
    "name": [f"Employee_{i}" for i in range(1, 6)],
    "role": np.random.choice(["Sales", "Support", "Manager"], size=5)
}))

WORK.add("stores", pd.DataFrame({
    "store_id": range(501, 504),
    "city": ["Stockholm", "Göteborg", "Malmö"],
    "manager": np.random.choice(WORK.get("employees")["name"], size=3)
}))

WORK.add("shipments", pd.DataFrame({
    "ship_id": range(601, 611),
    "order_id": np.random.choice(WORK.get("orders")["order_id"], size=10),
    "status": np.random.choice(["Pending", "Shipped", "Delivered"], size=10)
}))

WORK.add("returns", pd.DataFrame({
    "return_id": range(701, 706),
    "order_id": np.random.choice(WORK.get("orders")["order_id"], size=5),
    "reason": np.random.choice(["Damaged", "Wrong item", "Other"], size=5)
}))

# En joinad vy (som PROC SQL JOIN)
orders_full = (
    WORK.get("orders")
    .merge(WORK.get("customers"), on="cust_id", how="left")
    .merge(WORK.get("products"), on="prod_id", how="left")
)
WORK.add("orders_full", orders_full)

# 3) UI: vänster lista (WORK), höger visning (som SAS-flikar)
lst = w.Select(options=WORK.names(), rows=12, layout=w.Layout(width="260px"))
btn_head = w.Button(description="Visa head()", tooltip="Visa de första raderna")
btn_all  = w.Button(description="Visa hela", tooltip="Visa hela tabellen")
btn_over = w.Button(description="Översikt (alla)", tooltip="Visa WORK.overview()")
btn_del  = w.Button(description="Ta bort tabell", tooltip="Remove from WORK")
out = w.Output()

def refresh_list():
    lst.options = WORK.names()

def show_df(name, head_only=True):
    out.clear_output()
    with out:
        df = WORK.get(name)
        print(f"TABLE: {name}  |  shape={df.shape}")
        display(df.head() if head_only else df)

def on_select(change):
    if change["name"] == "value" and change["new"]:
        show_df(change["new"], head_only=True)

def on_head_clicked(_):
    if lst.value:
        show_df(lst.value, head_only=True)

def on_all_clicked(_):
    if lst.value:
        show_df(lst.value, head_only=False)

def on_over_clicked(_):
    out.clear_output()
    with out:
        print("WORK overview")
        display(WORK.overview())

def on_del_clicked(_):
    if lst.value:
        WORK.remove(lst.value)
        refresh_list()
        out.clear_output()

lst.observe(on_select, names="value")
btn_head.on_click(on_head_clicked)
btn_all.on_click(on_all_clicked)
btn_over.on_click(on_over_clicked)
btn_del.on_click(on_del_clicked)

controls = w.VBox([
    w.HTML("<b>WORK (tabeller)</b>"),
    lst,
    w.HBox([btn_head, btn_all]),
    w.HBox([btn_over, btn_del]),
])

ui = w.HBox([controls, out], layout=w.Layout(gap="16px"))
display(ui)

# Visa översikten initialt
on_over_clicked(None)
